# Исследование уровня потребительнской лояльности клиентов телекомунникационной компании

# Описание и задачи исследования

# Цель исследования

Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

# Источник данных

Мы будем работать с данными на основе опроса проведенного телекоммуникационной компанией, которая оказывает услуги на территории всего СНГ.

# Задачи исследования

1. Подключиться к базе, написать SQL-запрос и выгрузить данные.
2. Подготовить дашборд в Tableau с итогами исследования.
3. С помощью дашборда ответить на вопросы:
    1. Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
    2. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
    3. Какой общий NPS среди всех опрошенных?
    4. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

Импортируем все необходимые нам библиотеки:

In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

Выполним подключение к базе данных:

In [ ]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '****'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Напишем SQL-запрос в соответсвии с необоходимыми требованиями, который извлечет следующие данные:

`user_id` — идентификатор клиента;

`lt_day` — количество дней «жизни» клиента;

`is_new` — поле хранит информацию о том, является ли клиент новым;

`age` — возраст;

`gender_segment` — пол (для удобства работы с полем преобразуем значения в текстовый вид);

`os_name` — тип операционной системы;

`cpe_type_name` — тип устройства;

`country` — страна проживания;

`city` — город проживания;

`age_segment` — возрастной сегмент;

`traffic_segment` — сегмент по объёму потребляемого трафика;

`lifetime_segment` — сегмент по количеству дней «жизни»;

`nps_score` — оценка клиента в NPS-опросе;

`nps_group` — поле хранит информацию о том, к какой группе относится оценка клиента в опросе.

In [ ]:
query = """
SELECT user_id,
       lt_day,
       case
           when lt_day <= 365 then 'да'
           when lt_day > 365 then 'нет'
       end as is_new,
       age,
       case
           when gender_segment = 1 then 'женщина'
           when gender_segment = 0 then 'мужчина'
           else 'неизвестный'
       end as gender_segment,
       os_name,
       cpe_type_name,
       country,
       city,
       a.title as age_segment,
       ts.title as traffic_segment,
       ls.title as lifetime_segment,
       nps_score,
       case
           when nps_score >= 9 then 'cторонник'
           when nps_score = 8 OR nps_score = 7 then 'нейтрал'
           when nps_score <= 6 then 'критик'
       end as nps_group
FROM user
INNER JOIN location AS l ON user.location_id = l.location_id
INNER JOIN age_segment AS a ON user.age_gr_id = a.age_gr_id
INNER JOIN traffic_segment AS ts ON user.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON user.lt_gr_id = ls.lt_gr_id
"""

Создадим датафрейм и посмотрим на первые 3 строки:

In [ ]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,нет,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонник
1,A001WF,2344,нет,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонник
2,A003Q7,467,нет,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонник


Импортируем таблицу в `csv` файл и сохраним его для создания дашборда.

In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Итоги исследования

1. Больше всего клиентов находится в 3 (154 800) и 4 (165 933) сегментах. В каждом из этих сегментов примерно одинаковое соотношение женщин и мужчин, но общее число женщин больше – 272 442 чел. (54,22 %). Старых пользователей больше. По результатам опроса в топ-5 городов входят: Москва, Санкт-Петербург, Новосибирск, Екатеринбург и Казань.
2. Самые лояльные клиенты проживают в Иваново, девушки в возрасте от 35 до 44 лет наиболее лояльны, относящиеся к старым клиентам. Менее лояльные клиенты живут в Воронеже, группы – это люди в возрасте от 25 до 34 лет, при этом в Москве и Санкт-Петербурге преобладают мужчины.
3. Общий NPS среди всех опрошенных составил 21,9 %.
4. К сторонникам относятся люди в возрасте от 35 до 44 лет, проживающие в Москве, чаще всего это мужчины.

[Дашборд](https://public.tableau.com/views/telecomm_csi_dash/cis_nps?:language=en-US&:display_count=n&:origin=viz_share_link)

[Презентация](https://disk.yandex.ru/i/VxyVB5Qhh2VyLQ)